# Cài đặt các gói cần thiết

In [ ]:
%pip install onnx onnxscript onnxruntime
# pip install onnx onnxscript để có thể chuyển model từ .pth hoặc .pt sang .onnx
# pip install onnxruntime để có môi trường chạy model định dạng tệp .onnx

# Xây dựng model hoặc load vào model

In [ ]:
# model_file_path = '.../model.pt'
model_file_path = ...

In [ ]:
# Đây là ví dụ cách load vào model H97_ResNet đã được train và lưu vào file có đường dẫn model_file_path
from src.model.classifier.H97 import H97_ResNet
import torch

torch_model = H97_ResNet()
torch_model.load_state_dict(torch.load(model_file_path))

# Load model từ dạng của PyTorch sang format của ONNX

In [ ]:
torch_input = torch.randn(1, 1, 32, 32)
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)

# Lưu model ở format của ONNX thành file .onnx

In [ ]:
# model_destination_path = '.../my_image_classifier.onnx'
model_destination_path = ...

In [ ]:
onnx_program.save(model_destination_path)

# Load lại model từ file .onnx

In [ ]:
import onnx
onnx_model = onnx.load(model_destination_path)
onnx.checker.check_model(onnx_model)

# Trực quan kiến trúc model (lưu ở định dạng tệp .onnx) với [Netron](https://netron.app/)

Truy cập trang web [sau đây](https://netron.app/). Sau đó chọn tệp tin .onnx và xem hình ảnh trực quan ra.

# Chạy 1 model .onnx với môi trường onnxruntime

In [ ]:
import onnxruntime

torch_input = torch.randn(
    1, 1, 32, 32
)  # sửa lại kích thước input cho phù hợp với model của mình
onnx_input = onnx_program.adapt_torch_inputs_to_onnx(torch_input)
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

# Tạo 1 session để suy luận model
# tại đường dẫn model_destination_path
# với môi trường CPUExecutionProvider hoặc CUDAExecutionProvider
# Kiểm tra xem có GPU hỗ trợ ONNX Runtime không
if "CUDAExecutionProvider" in onnxruntime.get_available_providers():
    providers = ["CUDAExecutionProvider"]
else:
    providers = ["CPUExecutionProvider"]

# Tạo phiên suy luận sử dụng thiết bị phù hợp
ort_session = onnxruntime.InferenceSession(
    "./model_destination_path.onnx", providers=providers
)


def to_numpy(tensor):
    """
    Định nghĩa một hàm để chuyển tensor PyTorch sang mảng NumPy.
    Điều này cần thiết vì ONNX Runtime sử dụng mảng NumPy làm định dạng dữ liệu đầu vào.
    Nếu đang sử dụng GPU, tensor sẽ được chuyển về CPU trước khi chuyển sang NumPy.
    """
    if "CUDAExecutionProvider" in onnxruntime.get_available_providers():
        tensor = tensor.cuda()
    return (
        tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    )


# Tạo một từ điển của dữ liệu đầu vào cho ONNX Runtime,
# chuyển đổi tensor đầu vào PyTorch sang mảng NumPy và
# gán chúng với tên của các đầu vào mô hình như được xác định trong phiên suy luận ONNX.
onnxruntime_input = {
    k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)
}

# Thực hiện suy luận với dữ liệu đầu vào đã được chuẩn bị,
# None ở đây chỉ ra rằng chúng ta muốn lấy tất cả đầu ra từ mô hình.
# Kết quả suy luận được lưu trong onnxruntime_outputs.
onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

# So sánh kết quả chạy bởi model PyTorch với model .onnx có giống nhau không

In [ ]:
torch_outputs = torch_model(torch_input)
torch_outputs = onnx_program.adapt_torch_outputs_to_onnx(torch_outputs)

assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output, torch.tensor(onnxruntime_output))

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")